### I plan to use this file in order to get my head around the open quantum systems stuff. 
#### I plan it to go through all of the equations and theory, with included capturd images from papers etc when needed.
#### By the end I hope to be able to copy and paste from this into my project library

In [ ]:
import project_lib as mylib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib notebook

I think the most elementary thing to begin this task is the linblad master equation and it's approximations. The following is taken from the documentation of qutip

In [ ]:
# this is the code to copy and paste for an image
img1=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Jupyter Extra Images\linblad_master_equation.png")
plt.figure(); plt.imshow(img1)
plt.axis('off')
plt.show()

The Linblad Master Equation for the system coupled weakly with the environment is  
\begin{equation}
\dot{\rho}(t) = -\frac{i}{\hbar} \left[ H(t),\rho (t)\right]+\frac{1}{2}\sum_{k}^{} \left( 2C_{k}\rho (t)C_{k}^{\dagger}-\rho (t)C_{k}^{\dagger}C_{k}-C_{k}^{\dagger}C_{k} \rho (t) \right)
\end{equation}
where
\begin{equation}
C_{k} = \sqrt{\gamma_{k}} A_{k}
\end{equation}
The terms $C_{k}$ are the operators by which the qubit system couples to the environment. They ought to be the projection operators between states $\left|\psi_n\right>$ and $\left|\psi_m\right>$.
For now I think the most sensible thing to do might be to make the rate of each coupling dependent on the temperature, $T$, the energy gap, $\delta E_{nm} = E_{m}-E_{n} = -\delta E_{mn}$, and the number of spin flips (the Hamming distance), $d_nm = d_mn$. I will also set an overall parameter $\Gamma$ to which I can the global coupling strengths. Therefore the coupling operators are
\begin{equation}
C_{n \neq m} =  \sqrt{2^{\ -d_{nm}} \Gamma  exp \left( \frac{E_{n}-E_{m}}{k_{B}T} \right)} \left|\psi_m\right> \left<\psi_n\right|
\end{equation}
However, for every forward transfer there will be a backwards one
\begin{equation}
C_{m \neq n} =  \sqrt{ 2^{\ -d_{mn}} \Gamma exp \left( \frac{E_{m}-E_{n}}{k_{B}T} \right)} \left|\psi_n\right> \left<\psi_m\right|
\end{equation}

In [ ]:
#Here I want to make a function that can create the |n><m| term 

def make_eigen_vector(bits):
    '''this function takes a string of bit assignments and turns them into a vector in the hamiltonian space'''
    neg = np.matrix('0. 1.')
    pos = np.matrix('1. 0.')
    vecs = [pos,neg]
    vec = vecs[bits[0]]
    for i in range(1,len(bits)):
        vec = np.kron(vec,vecs[bits[i]])
    return vec

def projection_nm(nbits,mbits):
    '''this function takes 2 arrays of bit assignments and turns them into a projection operator in the joint hilbert 
    space of the 2 vectors.
    Essentially this creates |n><m|'''
    n = make_eigen_vector(nbits)
    m = make_eigen_vector(mbits)
    nm = np.kron(n,m.getH())
    return nm

nbits = [1,0,1]
mbits = [1,0,0]
n = make_eigen_vector(nbits)
m = make_eigen_vector(mbits)
print r'P_nm |n> = ',(projection_nm(nbits,mbits)*n.getH()).getH(),u"\u2020"
print r'     |m> = ', m ,u"\u2020"

In [ ]:
# I want to create a function that calculates the hamming distance between any bit assignments

def hamming_distance(nbits,mbits):
    nbits = np.asarray(nbits)
    mbits = np.asarray(mbits)
    d_nm = sum(np.absolute(nbits - mbits))
    return d_nm

nbits = [0,1,1,1]
mbits = [1,0,0,1]

print hamming_distance(nbits,mbits)

In [ ]:
# I want to make a function that will find the energy gap between an 2 bit assignments

def dE_nm(nbits,mbits,H):
    '''dE = E_m-E_n. This fucntion '''
    dE = 